In [1]:
# score per person per paragraph
import os
import time
from collections import defaultdict

import openai
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv

import utils
from utils import extract_score

label_to_score = utils.label_to_score

load_dotenv()
openai.api_key = os.getenv("OPENAI_KEY")

In [2]:
speaker_transcripts = pd.read_csv('../data/transcripts_byspeaker.csv')

In [3]:
prompt_template = """
<sentences>
INPUT
</sentences>
<labels>
Dovish: Strongly expresses a belief that the economy may be
growing too slowly and may need stimulus through mon-
etary policy.
Mostly dovish: Overall message expresses a belief that the economy may
be growing too slowly and may need stimulus through
monetary policy.
Neutral: Expresses neither a hawkish nor dovish view and is
mostly objective.
Mostly hawkish: Overall message expresses a belief that the economy is
growing too quickly and may need to be slowed down
through monetary policy.
Hawkish: Strongly expresses a belief that the economy is growing
too quickly and may need to be slowed down through monetary policy.
</labels>
Are sentences are all phrases said by a single speaker in a transcript. Which label best applies applies all of the sentences taken as a whole (Dovish, Mostly Dovish, Neutral, Mostly Hawkish, Hawkish)?
"""

In [4]:
from collections import defaultdict
import ast

scores = defaultdict(lambda: defaultdict(list))

In [5]:
for i, transcript in tqdm(speaker_transcripts.iterrows()):
    transcript_scores = scores[transcript["date"]]
    speakers_dict = ast.literal_eval(transcript["speaker_text"])
    for speaker in speakers_dict:
        speaker_scores = transcript_scores[speaker]
        for speaker_statement in speakers_dict[speaker]:
            if len(speaker_statement.split(" ")) < 6:
                speaker_scores.append("TOO SHORT")
                continue
            else:
                # call GPT-4
                response = None
                while response is None:
                    try:
                        response = openai.ChatCompletion.create(
                        model="gpt-4",
                        messages=[
                                {"role": "user", "content": prompt_template.replace("INPUT", speaker_statement)},
                            ],
                        temperature=0
                        )
                    except Exception as e:
                        print(e)
                        time.sleep(1)
                score = extract_score(response["choices"][0]["message"]["content"])
                speaker_scores.append(score)
        print(speaker, transcript["date"])

0it [00:00, ?it/s]

Rate limit reached for gpt-4 in organization org-Yjpq53VqHZJXMrSdJflswUXP on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues.
Rate limit reached for gpt-4 in organization org-Yjpq53VqHZJXMrSdJflswUXP on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues.
Rate limit reached for gpt-4 in organization org-Yjpq53VqHZJXMrSdJflswUXP on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues.
Rate limit reached for gpt-4 in organization org-Yjpq53VqHZJXMrSdJflswUXP on tokens per min. Limit: 10000 / min. Please try again in 6ms. Contact us through our help center at help.openai.com if you continue to have issues.
Rate limit reached for gpt-4 in organization org-Yjpq53VqHZJXMrSdJflswUXP on tokens per min. Limit: 1000

0it [04:49, ?it/s]


KeyboardInterrupt: 

In [ ]:
for i in scores[19940204]['CHAIRMAN GREENSPAN']:
    if i != 0 and i != 'TOO SHORT':
        print(i)

In [ ]:
scores[19940204]

In [ ]:
for i, transcript in (speaker_transcripts.iterrows()):
    transcript_scores = scores[transcript["date"]]
    speakers_dict = ast.literal_eval(transcript["speaker_text"])
    for speaker in speakers_dict:
        print(len(speakers_dict[speaker]))
        # break
    break

In [ ]:
scores = Counter(Counter(0))

In [ ]:
scores = {}
for i, transcript in tqdm(speaker_transcripts.iterrows()):
    print(transcript["speaker_text"])
    # speakers_dict = ast.literal_eval(transcript["speaker_text"])
    # print(speakers_dict)
    # for speaker in speakers_dict:
    #     print(speaker)
    break
    # text = statement['statements_text']
    # # call GPT-4
    # response = None
    # while response is None:
    #     try:
    #         response = openai.ChatCompletion.create(
    #         model="gpt-4",
    #         messages=[
    #                 {"role": "user", "content": prompt_template.replace("INPUT", text)},
    #             ],
    #         temperature=0
    #         )
    #     except Exception as e:
    #         print(e)
    #         time.sleep(1)

    # score = extract_score(response["choices"][0]["message"]["content"])

    # scores[statement["date"]] = score
    # print(scores)